In [16]:
import math
import metapy
import pytoml
import csv
import pandas as pd


class SearchEngine:
    def __init__(self, path, config):
        self.path = path
        self.idx = metapy.index.make_inverted_index(config)
        self.config = config
        self.query = metapy.index.Document()
        self.ranker = self.ranker(self.config)
        self.df = self.build_corpus()
        
    def build_corpus(self):
        
        try:
            df = pd.read_csv(self.path)
        except FileNotFoundError:
            print("Could not open/read file")
            
        textcolumn = ['content'] # ignore the other columns
        new_df_text = pd.DataFrame(df, columns=textcolumn)
        
        file = self.path[0:self.path.rfind('/') + 1]
        new_data_file = file + 'course.dat'
        with open(new_data_file, 'w') as f:
            for index, row in df.iterrows():
                text = row['content']
                f.write(text)
                f.write('\n')
        return df

    def ranker(self, config):
         return metapy.index.OkapiBM25(k1=1.65, b=0.75, k3=2.6)
        
    def ranker_score(self):
        ranker = self.ranker
        top_docs = ranker.score(self.idx, self.query, num_results=5)
        return top_docs

    def query_result(self,text):
        self.query.content(text) 
        top_docs = self.ranker_score()
        top_links = []
        for num, (d_id, _) in enumerate(top_docs):
            course = self.df.iloc[d_id-1].course_id
            week = self.df.iloc[d_id-1].week_nbr 
            lesson = self.df.iloc[d_id-1].video_id
            top_links.append((course,week,lesson)) # (course, week, lesson)
        return top_links

In [17]:
q1 = SearchEngine('/Users/kartikpatel/Desktop/CS 410/Final Project/CourseProject/courseera_video_lessons.csv', 'config.toml')
print(q1.query_result('What is PLSA?'))

[('cs-410', 9, 7), ('cs-410', 9, 6), ('cs-410', 12, 2), ('cs-410', 3, 3), ('cs-410', 9, 1)]
